In [1]:
import os; import sys ; import matplotlib as mpl ; import matplotlib.pyplot as plt ; import numpy as np
import pathlib; from pathlib import Path; import pandas as pd; import flopy

In [2]:
example_name = "MT3D_tutorial_07ii"
exe_name_mf = 'E:\\15_REPOS\\00_BETAMI\\bin\\mf2005'
exe_name_mt = 'E:\\15_REPOS\\00_BETAMI\\bin\\mt3d-usgs_1.1.0_64'
current_directory = os.getcwd();  print(current_directory)

E:\15_REPOS\00_BETAMI\98_Plus\02_MT3D


In [3]:
path = pathlib.Path('E:\\15_REPOS\\00_BETAMI\\98_Plus\\02_MT3D')
path.mkdir(parents=True, exist_ok=True); os.chdir(path); current_directory = os.getcwd()
directory_name = 'C_p07_V02_Iacopo'    ; workspace = os.path.join('.', directory_name)
if os.path.isdir(workspace) is False:
    os.mkdir(workspace) 
output_folder = Path('C_p07_V02_Iacopo') ; output_folder.mkdir(exist_ok=True)

In [4]:
def p07(dirname, mixelm):
    model_ws = os.path.join(workspace, dirname)
    nlay = 8 ;  nrow = 15 ;  ncol = 21 ;  delr = 10 ;  delc = 10 ;  delv = 10 ;  Lx = (ncol - 1) * delr ;  v = 1.0 / 3.0 ;  
    prsity = 0.2 ;  q = v * prsity ;  al = 10.0 ;  trpt = 0.3 ;  trpv = 0.3 ;  q0 = 0.5 ;  c0 = 100.0 ;  
    perlen_mf = 100.0 ;  perlen_mt = 100.0 ;  hk = 0.5 ;  laytyp = 0 ;  

    modelname_mf = dirname + "_mf"
    mf = flopy.modflow.Modflow(modelname=modelname_mf, model_ws=model_ws, exe_name=exe_name_mf)
    dis = flopy.modflow.ModflowDis(mf,nlay=nlay,nrow=nrow,ncol=ncol,delr=delr,delc=delc,top=0.0,
                                   botm=[-delv * k for k in range(1, nlay + 1)],perlen=perlen_mf)
    ibound = np.ones((nlay, nrow, ncol), dtype=int)                      ;  ibound [:, :, 0] = -1   ; ibound[:, :, -1] = -1
    strt = np.zeros((nlay, nrow, ncol), dtype=float)     ;  h1 = q * Lx  ;  strt   [:, :, 0] = h1
    bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=strt)
    lpf = flopy.modflow.ModflowLpf(mf, hk=hk, laytyp=laytyp)
    wel = flopy.modflow.ModflowWel(mf, stress_period_data=[[6, 7, 2, q0]])
    pcg = flopy.modflow.ModflowPcg(mf)
    lmt = flopy.modflow.ModflowLmt(mf)             ; mf.write_input()    ; mf.run_model(silent=True)

    modelname_mt = dirname + "_mt"
    mt = flopy.mt3d.Mt3dms(modelname=modelname_mt,model_ws=model_ws,exe_name=exe_name_mt,modflowmodel=mf)
    btn = flopy.mt3d.Mt3dBtn(mt, icbund=1, prsity=prsity, sconc=0)
    dceps = 1.0e-5 ;  nplane = 1 ;  npl = 0 ;  nph = 16 ;  npmin = 2 ;  npmax = 32 ;  dchmoc = 1.0e-3 ;  nlsink = nplane ;  npsink = nph   
    adv = flopy.mt3d.Mt3dAdv(mt,mixelm=mixelm,dceps=dceps,nplane=nplane,npl=npl,nph=nph,npmin=npmin,npmax=npmax,
                             nlsink=nlsink,npsink=npsink,percel=0.5)
    dsp = flopy.mt3d.Mt3dDsp(mt, al=al, trpt=trpt, trpv=trpv)          ;     spd = {0: [6, 7, 2, c0, 2]}
    ssm = flopy.mt3d.Mt3dSsm(mt, stress_period_data=spd)
    gcg = flopy.mt3d.Mt3dGcg(mt)                                       ;     mt.write_input()
    fname = os.path.join(model_ws, "MT3D001.UCN")
    if os.path.isfile(fname):
        os.remove(fname)
    mt.run_model(silent=True)
    
    fname = os.path.join(model_ws, "MT3D001.UCN") ; ucnobj = flopy.utils.UcnFile(fname) ; times = ucnobj.get_times() ; conc = ucnobj.get_alldata()
    fname = os.path.join(model_ws, "MT3D001.OBS")
    if os.path.isfile(fname):
        cvt = mt.load_obs(fname)
    else:
        cvt = None
    fname = os.path.join(model_ws, "MT3D001.MAS")  ;     mvt = mt.load_mas(fname)
    return mf, mt, conc, cvt, mvt

In [5]:
fig = plt.figure(figsize=(5, 10))  ; grid = mf.modelgrid  ; 
mf, mt, conc, cvt, mvt = p07("p07", -1)

conc = conc[0]
ax = fig.add_subplot(3, 1, 1, aspect="equal") ; ilay = 4
pmv = flopy.plot.PlotMapView(ax=ax, model=mf, layer=ilay)  ; pmv.plot_grid(color=".5", alpha=0.2) ; pmv.plot_ibound()
cs = pmv.contour_array(conc, levels=[0.01, 0.05, 0.15, 0.50], colors="k")  ; plt.clabel(cs)
plt.xlabel("DISTANCE X-AXIS (m)") ; plt.ylabel("DISTANCE Y-AXIS (m)")      ; plt.title("LAYER {}".format(ilay + 1))
ax = fig.add_subplot(3, 1, 2, aspect="equal") ; ilay = 5
pmv = flopy.plot.PlotMapView(ax=ax, model=mf, layer=ilay)  ; pmv.plot_grid(color=".5", alpha=0.2) ; pmv.plot_ibound()
cs = pmv.contour_array(conc, levels=[0.01, 0.05, 0.15, 0.50], colors="k")  ; plt.clabel(cs)
plt.xlabel("DISTANCE  X-AXIS (m)") ; plt.ylabel("DISTANCE  Y-AXIS (m)")    ; plt.title("LAYER {}".format(ilay + 1))
ax = fig.add_subplot(3, 1, 3, aspect="equal")  ; ilay = 6
pmv = flopy.plot.PlotMapView(ax=ax, model=mf, layer=ilay)  ; pmv.plot_grid(color=".5", alpha=0.2)  ; pmv.plot_ibound()
cs = pmv.contour_array(conc, levels=[0.01, 0.05, 0.15, 0.50], colors="k")  ; plt.clabel(cs)
plt.xlabel("DISTANCE X-AXIS (m)") ; plt.ylabel("DISTANCE Y-AXIS (m)")      ; plt.title("LAYER {}".format(ilay + 1))
plt.plot(grid.xcellcenters[7, 2], grid.ycellcenters[7, 2], "ko")           ; plt.tight_layout()

C:\Users\gross.DESKTOP-AIRQM2M\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
C:\Users\gross.DESKTOP-AIRQM2M\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
C:\Users\gross.DESKTOP-AIRQM2M\anaconda3\Lib\ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)


NameError: name 'mf' is not defined

<Figure size 500x1000 with 0 Axes>

In [ ]:
print(conc.shape),  type(conc)